In [15]:
import ollama
import pandas as pd
import logging
from typing import Optional, Dict, Any, List
import time

# logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('llm_evaluation.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

try:
    df = pd.read_csv("data/Farming_FAQ_Assistant_Dataset.csv")  # FAQ_farming_dataset.csv
    logger.info(f"Successfully loaded dataset with {len(df)} questions")
except FileNotFoundError:
    logger.error("Error: Farming_FAQ_Assistant_Dataset.csv not found. Make sure the file exists.")
    exit()

# send prompt to Ollama
def get_llm_answer_ollama(prompt: str) -> Optional[str]:
    """
    Send a prompt to Ollama and get the response.
    
    Args:
        prompt: The question/prompt to send to the LLM
        
    Returns:
        The LLM's response as a string, or None if there was an error
    """
    try:
        response = ollama.generate(
            model='llama3.2:3b',
            prompt=prompt
        )
        
        llm_response = response['response'].strip()
        
        # Print the LLM response
        print(f"\n=== LLM RESPONSE ===")
        print(f"Prompt: {prompt[:100]}...")
        print(f"Response: {llm_response}")
        print("=" * 50)
        
        return llm_response
    except Exception as e:
        logger.error(f"Error communicating with Ollama: {e}")
        return None

# zero-shot prompt
def create_zero_shot_prompt(question: str) -> str:
    """
    Create a zero-shot prompt for farming questions.
    
    Args:
        question: The farming question to be answered
        
    Returns:
        A formatted prompt string
    """
    prompt = f"""You are an expert agricultural assistant. Answer the following farming question based on your knowledge. Provide a concise, practical answer.

Question: {question}
Answer:"""
    return prompt

# 4. evaluate answer similarity
def evaluate_answer_similarity(llm_answer: str, ground_truth: str) -> bool:
    """
    Evaluate if the LLM answer is similar to the ground truth.
    
    Args:
        llm_answer: The LLM's response
        ground_truth: The expected correct answer
        
    Returns:
        True if answers are similar enough, False otherwise
    """
    # simple keyword-based similarity check
    llm_words = set(llm_answer.lower().split())
    truth_words = set(ground_truth.lower().split())
    
    # calculate Jaccard similarity
    intersection = len(llm_words.intersection(truth_words))
    union = len(llm_words.union(truth_words))
    
    similarity = intersection / union if union > 0 else 0
    return similarity > 0.3  # Threshold can be adjusted

# evaluate the answers
correct_answers: List[Dict[str, Any]] = []
incorrect_answers: List[Dict[str, Any]] = []

logger.info("Starting LLM evaluation...")

for index, row in df.iterrows():
    question = row["Question"]
    ground_truth = row["Answer"]
    
    logger.info(f"Question {index + 1}: Processing...")
    
    prompt = create_zero_shot_prompt(question)
    llm_answer = get_llm_answer_ollama(prompt)
    
    if llm_answer:
        is_correct = evaluate_answer_similarity(llm_answer, ground_truth)
        
        result_entry = {
            "Question": question,
            "Ground Truth": ground_truth,
            "LLM Answer": llm_answer,
            "Correct": is_correct
        }
        
        if is_correct:
            correct_answers.append(result_entry)
            logger.info(f"Question {index + 1}: ✓ Answered correctly")
        else:
            incorrect_answers.append(result_entry)
            logger.info(f"Question {index + 1}: ✗ Answered incorrectly")
    else:
        logger.error(f"Question {index + 1}: Failed to get response from LLM")
        incorrect_answers.append({
            "Question": question,
            "Ground Truth": ground_truth,
            "LLM Answer": "No response",
            "Correct": False
        })
    
    # add a small delay to avoid overwhelming the local Ollama server
    time.sleep(0.5)

# calculate and log final statistics
total_questions = len(df)
correct_count = len(correct_answers)
accuracy = (correct_count / total_questions) * 100 if total_questions > 0 else 0

logger.info(f"\n=== EVALUATION RESULTS ===")
logger.info(f"Total Questions: {total_questions}")
logger.info(f"Correct Answers: {correct_count}")
logger.info(f"Incorrect Answers: {len(incorrect_answers)}")
logger.info(f"Accuracy: {accuracy:.2f}%")

# print results
print("\n=== CORRECT ANSWERS ===")
for i, answer in enumerate(correct_answers[:5]):  # Show first 5
    print(f"{i+1}. Q: {answer['Question'][:50]}...")
    print(f"   A: {answer['LLM Answer'][:100]}...")
    print()

print("\n=== INCORRECT ANSWERS ===")
for i, answer in enumerate(incorrect_answers[:5]):  # Show first 5
    print(f"{i+1}. Q: {answer['Question'][:50]}...")
    print(f"   Expected: {answer['Ground Truth'][:100]}...")
    print(f"   Got: {answer['LLM Answer'][:100]}...")
    print()

# save results to CSV
results_df = pd.DataFrame(correct_answers + incorrect_answers)
results_df.to_csv("data/llm_eval/ollama_zero_shot_results.csv", index=False)
logger.info("Results saved to ollama_zero_shot_results.csv")

2025-06-30 18:35:35,200 - INFO - Successfully loaded dataset with 500 questions
2025-06-30 18:35:35,201 - INFO - Starting LLM evaluation...
2025-06-30 18:35:35,203 - INFO - Question 1: Processing...
2025-06-30 18:35:37,297 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-06-30 18:35:37,299 - INFO - Question 1: ✗ Answered incorrectly



=== LLM RESPONSE ===
Prompt: You are an expert agricultural assistant. Answer the following farming question based on your knowle...
Response: The ideal time to plant rice varies depending on the region and climate, but generally, it's best to plant rice during the wet season when temperatures are warm (above 75°F/24°C) and rainfall is abundant. In tropical regions, this usually occurs between May and September, while in temperate climates, it's typically planted in late spring to early summer.

In general, the optimal planting time for rice is:

* 2-4 weeks after the last frost
* When the soil temperature reaches 64°F (18°C) to 85°F (29°C)
* Before the onset of dry weather or extreme temperatures

It's also essential to consider factors like sunlight, water availability, and soil type when determining the best planting time for rice in your specific region.


2025-06-30 18:35:37,813 - INFO - Question 2: Processing...
2025-06-30 18:35:39,616 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-06-30 18:35:39,617 - INFO - Question 2: ✗ Answered incorrectly



=== LLM RESPONSE ===
Prompt: You are an expert agricultural assistant. Answer the following farming question based on your knowle...
Response: Watering frequency for tomatoes depends on weather conditions and soil type. Here's a general guideline:

1. **Check soil moisture**: Stick your finger into the soil up to the first knuckle. If it feels dry, it's time to water.
2. **Water deeply but infrequently**: Tomatoes need about 1-2 inches of water per week. Water plants thoroughly, allowing excess water to drain away from the roots.
3. **Adjust for weather**:
 * Hot and dry weather: Water every other day or daily if possible.
 * Cool and rainy weather: Reduce watering frequency to once a week.

Remember, overwatering can lead to root rot and other issues.


2025-06-30 18:35:40,131 - INFO - Question 3: Processing...
2025-06-30 18:35:41,721 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-06-30 18:35:41,722 - INFO - Question 3: ✗ Answered incorrectly


...


=== LLM RESPONSE ===
Prompt: You are an expert agricultural assistant. Answer the following farming question based on your knowle...
Response: Watering frequency for tomato plants depends on weather conditions. Here's a general guideline:

1. **Check soil moisture**: Stick your finger into the soil up to the first knuckle. If it feels dry, it's time to water.
2. **Water deeply but infrequently**: Tomatoes need about 1-2 inches of water per week. Water at the base of the plant, avoiding leaves to prevent fungal diseases.
3. **Adjust for weather**:
 * In hot and dry weather, water every 2-3 days.
 * In cool and rainy weather, reduce watering to once a week.

Remember to avoid overwatering, which can lead to root rot and other issues.


2025-06-30 18:48:48,678 - INFO - Question 238: Processing...
2025-06-30 18:48:50,755 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-06-30 18:48:50,755 - INFO - Question 238: ✗ Answered incorrectly



=== LLM RESPONSE ===
Prompt: You are an expert agricultural assistant. Answer the following farming question based on your knowle...
Response: For wheat crops, NPK (Nitrogen-Phosphorus-Potassium) fertilizers are commonly recommended. However, considering wheat's specific needs:

* **Nitrogen (N)**: Promotes healthy leaf growth and yields a more vigorous crop.
* **Phosphorus (P)**: Encourages root development, grain production, and flower formation.

The ideal fertilizer for wheat crops is often a balanced N-P-K ratio of 90-60-30. 

A specific type of fertilizer that's beneficial for wheat is Potash fertilizer with 0-0-100 K. This provides potassium, which helps regulate water balance within the plant, improving drought tolerance and reducing nutrient deficiencies.

Additionally, consider applying organic fertilizers like manure or compost to supplement wheat crops.


2025-06-30 18:48:51,263 - INFO - Question 239: Processing...


KeyboardInterrupt: 